In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter



wine=pd.read_csv("wine/wine.csv")
wine
wine_review= wine[(wine['category'] == 'Red') | (wine['category'] == 'White') | (wine['category'] == 'Sparkling')]
wine_review.drop(columns= ['wine' , 'winery', 'designation' , 'varietal' , 'appellation' , 'alcohol' , 'price' , 'rating' , 'reviewer'])


nltk.download('averaged_perceptron_tagger')



categories = ['Red', 'White', 'Sparkling']

# creo un nuovo DataFrame vuoto
new_df = pd.DataFrame(columns=['category', 'word', 'count'])

# ciclo su tutte le categorie di interesse
for category in categories:

    category_wine = wine_review[wine_review['category'] == category]
    
    # unisco tutte le stringhe della colonna in un'unica stringa e poi la converto in una lista
    text = ' '.join(category_wine['review'].astype(str).tolist())
    words = text.split()
    # rimuovo la punteggiatura e le stopwords (parole che potrebbero alterare il risultato dell'analisi: articoli,ecc)
    stop_words = set(stopwords.words('english'))
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    # conta le occorrenze delle parole
    word_counts = Counter(words)
    top_words = word_counts.most_common(20)
    
    # DataFrame temporaneo e lo aggiungo al nuovp
    temp_df = pd.DataFrame(top_words, columns=['word', 'count'])
    temp_df['category'] = category
    new_df = new_df.append(temp_df, ignore_index=True)

new_df


# Eseguo il part-of-speech tagging delle parole
pos_tags = nltk.pos_tag(new_df['word'])

# Creo un dataframe vuoto per contenere le parole con etichetta uguale a "JJ" (!=aggettivi)
df_final = pd.DataFrame(columns=['word'])

for i, (parola, pos) in enumerate(pos_tags):
    if pos == 'JJ' :
        df_final = df_final.append({'word': parola}, ignore_index=True)
        
df_final = pd.merge(df_final, new_df, on='word')
df_final.drop_duplicates()
df_final

# analisi per vedere se le review contengono sinonimi delle parole più comuni trovate
nltk.download('omw-1.4')
from nltk.corpus import wordnet
nltk.download('wordnet')

df_final['word'] = df_final['word'].str.lower()

synonyms = []
for word in df_final['word']:
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
synonyms = set(synonyms)
synonyms = [word for word in synonyms if word not in wine['review'].tolist()]
print(synonyms)

# controllo nelle review se sono presenti i sinonimi trovati
result_list = []
for el in synonyms:
    sum_el = wine.review.str.contains(r'\b{}\b'.format(el), regex= True, case=False).sum()
    result_list.append((el, sum_el))

df = pd.DataFrame(result_list, columns=['el', 'sum_el'])
print(df)